# Trace Clustering in Process Mining

In [1]:

## Importing librarires
import pm4py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from pm4py.objects.log.util import get_log_representation as get_
warnings.simplefilter("ignore")
from sklearn.decomposition import PCA
from pm4py.objects.log.obj import EventLog, Trace
import matplotlib.pyplot as plt
# from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from pm4py.objects.log.log import EventLog



## Importing log
Importing event log of Sepsis

In [2]:
# Original event log

# Importing a XES event log
from pm4py.objects.log.importer.xes import importer as xes_importer
log = xes_importer.apply('/Users/asadtariq/Downloads/mimicel-mimic-iv-event-log-for-emergency-department-1.0.0/data/mimicel.xes')
#log[1],type(log)

parsing log, completed traces ::  13%|█▎        | 58247/448972 [07:05<53:56, 120.72it/s]  

### Filtered log to reduce size and set the containers

### Extracting total Activities

In [ ]:
len(log)

448972

In [ ]:
log[0]

{'attributes': {'concept:name': '35341790', 'subject_id': 13238787, 'hadm_id': nan, 'acuity': 3.0, 'chiefcomplaint': 'Back pain, RESTLESSNESS'}, 'events': [{'timestamp': datetime.datetime(2110, 1, 11, 1, 45, tzinfo=datetime.timezone.utc), 'activity': 'Enter the ED', 'seq_num': nan, 'icd_code': nan, 'icd_version': nan, 'icd_title': nan, 'temperature': nan, 'heartrate': nan, 'resprate': nan, 'o2sat': nan, 'sbp': nan, 'dbp': nan, 'pain': nan, 'rhythm': nan, 'name': nan, 'gsn': nan, 'ndc': nan, 'etc_rn': nan, 'etccode': nan, 'etcdescription': nan, 'med_rn': nan, 'gsn_rn': nan, 'case:concept:name': '35341790', 'case:subject_id': 13238787, 'case:hadm_id': nan, 'case:acuity': 3.0, 'case:chiefcomplaint': 'Back pain, RESTLESSNESS'}, '..', {'timestamp': datetime.datetime(2110, 1, 11, 6, 57, tzinfo=datetime.timezone.utc), 'activity': 'Medicine dispensations', 'seq_num': nan, 'icd_code': nan, 'icd_version': nan, 'icd_title': nan, 'temperature': nan, 'heartrate': nan, 'resprate': nan, 'o2sat': nan,

In [ ]:
from pm4py.algo.filtering.log.attributes import attributes_filter
activities = attributes_filter.get_attribute_values(log, "activity")
activities

{'Enter the ED': 448972,
 'Triage in the ED': 448972,
 'Vital sign check': 1651119,
 'Discharge from the ED + diagnosis 1': 447780,
 'Discharge from the ED + diagnosis 2': 270661,
 'Medicine dispensations': 1674652,
 'Discharge from the ED + diagnosis 3': 134979,
 'Medicine reconciliation': 3147294,
 'Discharge from the ED + diagnosis 4': 59435,
 'Discharge from the ED + diagnosis 5': 23872,
 'Discharge from the ED + diagnosis 7': 2738,
 'Discharge from the ED + diagnosis 6': 8719,
 'Discharge from the ED + diagnosis 8': 790,
 'Discharge from the ED + diagnosis 9': 198}

In [ ]:
len(activities)

14

In [ ]:

### Converting activity keys in to a list
activities_array = activities.keys()
activities_array = list(activities_array)
activities_array


['Enter the ED',
 'Triage in the ED',
 'Vital sign check',
 'Discharge from the ED + diagnosis 1',
 'Discharge from the ED + diagnosis 2',
 'Medicine dispensations',
 'Discharge from the ED + diagnosis 3',
 'Medicine reconciliation',
 'Discharge from the ED + diagnosis 4',
 'Discharge from the ED + diagnosis 5',
 'Discharge from the ED + diagnosis 7',
 'Discharge from the ED + diagnosis 6',
 'Discharge from the ED + diagnosis 8',
 'Discharge from the ED + diagnosis 9']

## Converting log in to a pandas dataframe

In [ ]:
len(log)

9926

In [ ]:

from pm4py.objects.conversion.log import converter as log_converter
dataframe = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
#dataframe.to_csv('log.csv')
#dataframe
## Trace profiling on Activities
dataframe =dataframe[['case:concept:name','activity']]
dataframe

,case:concept:name,activity
0,35341790,Enter the ED
1,35341790,Triage in the ED
2,35341790,Vital sign check
3,35341790,Discharge from the ED + diagnosis 1
4,35341790,Discharge from the ED + diagnosis 2
...,...,...
8320176,33494247,Vital sign check
8320177,33494247,Vital sign check
8320178,33494247,Discharge from the ED + diagnosis 3
8320179,33494247,Discharge from the ED + diagnosis 2


### Making np_array from Dataframe and grouping them on the basis of case identifier

Events in each trace of the complete event log.

In [ ]:
dataframe.to_csv("dataframe_activities_filtered.csv")

In [ ]:
len(np_arr)

8320181

In [ ]:
len(events)

9926

In [ ]:
len(events[1])

18

In [ ]:
dataframe

,case:concept:name,activity
0,30000177,Enter the ED
1,30000177,Triage in the ED
2,30000177,Medicine reconciliation
3,30000177,Medicine reconciliation
4,30000177,Medicine reconciliation
...,...,...
189780,36011667,Medicine reconciliation
189781,36011667,Medicine reconciliation
189782,36011667,Medicine reconciliation
189783,36011667,Vital sign check


In [ ]:
df = dataframe.groupby('case:concept:name')['activity'].apply(list)

In [ ]:
df.head()

case:concept:name
30000177    [Enter the ED, Triage in the ED, Medicine reco...
30000443    [Enter the ED, Triage in the ED, Vital sign ch...
30000573    [Enter the ED, Triage in the ED, Medicine reco...
30000679    [Enter the ED, Triage in the ED, Medicine reco...
30002837    [Enter the ED, Triage in the ED, Vital sign ch...
Name: activity, dtype: object

In [ ]:
events = df.values
type(events)

numpy.ndarray

In [ ]:
activities_array

['Enter the ED',
 'Triage in the ED',
 'Vital sign check',
 'Discharge from the ED + diagnosis 1',
 'Discharge from the ED + diagnosis 2',
 'Medicine dispensations',
 'Discharge from the ED + diagnosis 3',
 'Medicine reconciliation',
 'Discharge from the ED + diagnosis 4',
 'Discharge from the ED + diagnosis 5',
 'Discharge from the ED + diagnosis 7',
 'Discharge from the ED + diagnosis 6',
 'Discharge from the ED + diagnosis 8',
 'Discharge from the ED + diagnosis 9']

In [ ]:
activities_array = ['Vital sign check', 'Medicine dispensations', 'Medicine reconciliation']
activities_array


['Vital sign check', 'Medicine dispensations', 'Medicine reconciliation']

In [ ]:
length  = len(log)
width = len(activities_array)
length,width
vectors  = [ [0] * width for _ in range(length)]
type(vectors),len(vectors)
vectors

[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0,

In [ ]:
for i in range(len(events)):
        for j in range(len(events[i])):
                if events[i][j] == activities_array[0]:
                    vectors[i][0] += 1
                elif events[i][j] == activities_array[1]:
                    vectors[i][1] += 1
                elif events[i][j] == activities_array[2]:
                    vectors[i][2] += 1
                # elif events[i][j] == activities_array[3]:
                #     vectors[i][3] += 1
                # elif events[i][j] == activities_array[4]:
                #     vectors[i][4] += 1
                # elif events[i][j] == activities_array[5]:
                #     vectors[i][5] += 1
                # elif events[i][j] == activities_array[6]:
                #     vectors[i][6] += 1
                # elif events[i][j] == activities_array[7]:
                #     vectors[i][7] += 1
                # elif events[i][j] == activities_array[8]:
                #     vectors[i][8] += 1
                # elif events[i][j] == activities_array[9]:
                #     vectors[i][9] += 1
                # elif events[i][j] == activities_array[10]:
                #     vectors[i][10] += 1
                # elif events[i][j] == activities_array[11]:
                #     vectors[i][11] += 1
                # elif events[i][j] == activities_array[12]:
                #     vectors[i][12] += 1
                # elif events[i][j] == activities_array[13]:
                #     vectors[i][13] += 1

In [ ]:
len(vectors)

9926

In [ ]:
vectors

[[3, 3, 13],
 [1, 6, 0],
 [3, 2, 11],
 [6, 3, 2],
 [0, 0, 1],
 [5, 1, 6],
 [3, 2, 0],
 [1, 6, 15],
 [3, 13, 6],
 [4, 8, 8],
 [7, 5, 7],
 [3, 3, 2],
 [6, 0, 14],
 [3, 7, 21],
 [5, 2, 1],
 [3, 1, 0],
 [5, 4, 25],
 [11, 5, 9],
 [1, 1, 4],
 [2, 0, 31],
 [2, 0, 20],
 [3, 2, 15],
 [2, 0, 12],
 [2, 2, 18],
 [2, 3, 0],
 [13, 9, 16],
 [2, 7, 17],
 [4, 0, 0],
 [4, 0, 9],
 [7, 2, 10],
 [2, 14, 11],
 [3, 0, 0],
 [4, 0, 3],
 [3, 1, 6],
 [5, 15, 23],
 [3, 6, 1],
 [4, 13, 10],
 [2, 1, 4],
 [3, 1, 0],
 [2, 2, 0],
 [2, 2, 0],
 [5, 0, 5],
 [5, 1, 0],
 [1, 0, 1],
 [4, 1, 7],
 [2, 4, 17],
 [1, 1, 2],
 [1, 0, 0],
 [0, 1, 15],
 [2, 4, 0],
 [2, 0, 1],
 [4, 3, 0],
 [3, 10, 0],
 [2, 0, 6],
 [5, 2, 2],
 [2, 2, 7],
 [1, 1, 0],
 [1, 1, 7],
 [4, 8, 11],
 [3, 0, 1],
 [8, 11, 1],
 [6, 5, 18],
 [21, 15, 5],
 [1, 2, 12],
 [5, 11, 6],
 [2, 0, 0],
 [2, 2, 16],
 [2, 0, 12],
 [8, 6, 1],
 [4, 2, 17],
 [2, 0, 0],
 [4, 5, 14],
 [4, 9, 0],
 [4, 5, 21],
 [5, 0, 13],
 [4, 0, 28],
 [3, 3, 8],
 [2, 0, 20],
 [3, 2, 7],
 [0, 0, 0],

##  Creating Set of Activities - Extracting Activities from the Event log

In [ ]:
# set_vector, features = get_.get_representation(log, str_ev_attr=['activity'], str_tr_attr=[],
#                                                   num_ev_attr=[], num_tr_attr=[], str_evsucc_attr=[])

In [ ]:
# set_vector.shape

(9926, 14)

In [ ]:
set_vector, features = get_.get_representation(log, str_ev_attr=['activity'], str_tr_attr=[],
                                                  num_ev_attr=[], num_tr_attr=[], str_evsucc_attr=[])

In [ ]:
len(set_vector)

448972

### Merging Set and Bag vectors

In [ ]:
features

['event:activity@Discharge from the ED + diagnosis 1',
 'event:activity@Discharge from the ED + diagnosis 2',
 'event:activity@Discharge from the ED + diagnosis 3',
 'event:activity@Discharge from the ED + diagnosis 4',
 'event:activity@Discharge from the ED + diagnosis 5',
 'event:activity@Discharge from the ED + diagnosis 6',
 'event:activity@Discharge from the ED + diagnosis 7',
 'event:activity@Discharge from the ED + diagnosis 8',
 'event:activity@Discharge from the ED + diagnosis 9',
 'event:activity@Enter the ED',
 'event:activity@Medicine dispensations',
 'event:activity@Medicine reconciliation',
 'event:activity@Triage in the ED',
 'event:activity@Vital sign check']

In [ ]:
set_vector[0]

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1])

In [ ]:
vectors[0]

[3, 3, 13]

In [ ]:
merged = np.concatenate((vectors, set_vector), axis=1)

In [ ]:
merged[0]

array([ 3,  3, 13,  1,  1,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  1,  1])

In [ ]:
kmeans = KMeans(
        init="k-means++",
        n_clusters=10,
        n_init=10,
        max_iter=500,
        random_state=42
    )
km = kmeans.fit(merged)

already_seen = {}
labels = km.labels_
clusters = []

for i in range(len(log)):
        if not labels[i] in already_seen:
            already_seen[labels[i]] = len(list(already_seen.keys()))
            clusters.append(EventLog())
        trace = log[i]
        clusters[already_seen[labels[i]]].append(trace)




In [ ]:
silhouette_avg1 = silhouette_score(merged, kmeans.labels_)

In [ ]:
for i in range(len(clusters)):
    print(len(clusters[i]))

51211
47809
19140
134401
57329
33972
17124
28252
19641
19011
5163
5356
655
4876
5032


In [ ]:
sub_log1 = clusters[0]
sub_log2 = clusters[1]
sub_log3 = clusters[2]
sub_log4 = clusters[3]
sub_log5 = clusters[4]
sub_log6 = clusters[5]
sub_log7 = clusters[6]
sub_log8 = clusters[7]
sub_log9 = clusters[8]
sub_log10 = clusters[9]


In [ ]:
pm4py.write_xes(sub_log1, 'Mimic_sublogs_normal/sublog1.xes')
pm4py.write_xes(sub_log2, 'Mimic_sublogs_normal/sublog2.xes')
pm4py.write_xes(sub_log3, 'Mimic_sublogs_normal/sublog3.xes')
pm4py.write_xes(sub_log4, 'Mimic_sublogs_normal/sublog4.xes')
pm4py.write_xes(sub_log5, 'Mimic_sublogs_normal/sublog5.xes')
pm4py.write_xes(sub_log6, 'Mimic_sublogs_normal/sublog6.xes')
pm4py.write_xes(sub_log7, 'Mimic_sublogs_normal/sublog7.xes')
pm4py.write_xes(sub_log8, 'Mimic_sublogs_normal/sublog8.xes')
pm4py.write_xes(sub_log9, 'Mimic_sublogs_normal/sublog9.xes')
pm4py.write_xes(sub_log10, 'Mimic_sublogs_normal/sublog10.xes')

exporting log, completed traces :: 100%|██████████| 4/4 [00:00<00:00, 234.54it/s]


## PCA - Dimension Reduction. 
Transforming data in to 3 dimensional data.

In [40]:
merged.shape

(448972, 17)

In [ ]:
pca = PCA(n_components=3)
pca.fit(merged)
data3d = pca.transform(merged)
   

##  Elbow Method to extract optimal number of clusters

In [ ]:




kmeans = KMeans(
        init="random",
        n_clusters=10,
        n_init=10,
        max_iter=300,
        random_state=42
    )
km = kmeans.fit(data3d)

already_seen = {}
labels = km.labels_
clusters = []

for i in range(len(log)):
        if not labels[i] in already_seen:
            already_seen[labels[i]] = len(list(already_seen.keys()))
            clusters.append(EventLog())
        trace = log[i]
        clusters[already_seen[labels[i]]].append(trace)

type(clusters[0]),len(clusters[0]),len(clusters[1]),len(clusters[2])


(pm4py.objects.log.obj.EventLog, 58848, 53184, 161096)

In [ ]:
for i in range(len(clusters)):
    print(len(clusters[i]))

58848
53184
161096
80624
14001
31906
16088
9775
20765
2685


In [ ]:
sub_log1 = clusters[0]
sub_log2 = clusters[1]
sub_log3 = clusters[2]
sub_log4 = clusters[3]
sub_log5 = clusters[4]
sub_log6 = clusters[5]
sub_log7 = clusters[6]
sub_log8 = clusters[7]
sub_log9 = clusters[8]
sub_log10 = clusters[9]



In [ ]:
silhouette_avg2 = silhouette_score(merged, kmeans.labels_)

In [ ]:
pm4py.write_xes(sub_log1, 'Mimic_sublogs_PCA/sublog1.xes')
pm4py.write_xes(sub_log2, 'Mimic_sublogs_PCA/sublog2.xes')
pm4py.write_xes(sub_log3, 'Mimic_sublogs_PCA/sublog3.xes')
pm4py.write_xes(sub_log4, 'Mimic_sublogs_PCA/sublog4.xes')
pm4py.write_xes(sub_log5, 'Mimic_sublogs_PCA/sublog5.xes')
pm4py.write_xes(sub_log6, 'Mimic_sublogs_PCA/sublog6.xes')
pm4py.write_xes(sub_log7, 'Mimic_sublogs_PCA/sublog7.xes')
pm4py.write_xes(sub_log8, 'Mimic_sublogs_PCA/sublog8.xes')
pm4py.write_xes(sub_log9, 'Mimic_sublogs_PCA/sublog9.xes')
pm4py.write_xes(sub_log10, 'Mimic_sublogs_PCA/sublog10.xes')

exporting log, completed traces :: 100%|██████████| 3/3 [00:00<00:00, 823.97it/s]


In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(merged)
descriptive_features_normailsed = pd.DataFrame(x_scaled)
descriptive_features_normailsed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.027523,0.015544,0.122642,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.009174,0.031088,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.027523,0.010363,0.103774,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,0.055046,0.015544,0.018868,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.000000,0.000000,0.009434,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


##  Elbow Method to extract optimal number of clusters

In [ ]:




kmeans = KMeans(
        init="k-means++",
        n_clusters=10,
        n_init=10,
        max_iter=500,
        random_state=42
    )
km = kmeans.fit(descriptive_features_normailsed)

already_seen = {}
labels = km.labels_
clusters = []

for i in range(len(log)):
        if not labels[i] in already_seen:
            already_seen[labels[i]] = len(list(already_seen.keys()))
            clusters.append(EventLog())
        trace = log[i]
        clusters[already_seen[labels[i]]].append(trace)


In [ ]:
silhouette_avg3 = silhouette_score(merged, kmeans.labels_)

In [ ]:
for i in range(len(clusters)):
    print(len(clusters[i]))

25721
24365
42461
17446
69430
23872
21776
21262
94223
28970
30753
25008
23685


In [ ]:
sub_log1 = clusters[0]
sub_log2 = clusters[1]
sub_log3 = clusters[2]
sub_log4 = clusters[3]
sub_log5 = clusters[4]
sub_log6 = clusters[5]
sub_log7 = clusters[6]
sub_log8 = clusters[7]
sub_log9 = clusters[8]
sub_log10 = clusters[9]

In [ ]:
pm4py.write_xes(sub_log1, 'Mimic_sublogs_normalalised/sublog1.xes')
pm4py.write_xes(sub_log2, 'Mimic_sublogs_normalalised/sublog2.xes')
pm4py.write_xes(sub_log3, 'Mimic_sublogs_normalalised/sublog3.xes')
pm4py.write_xes(sub_log4, 'Mimic_sublogs_normalalised/sublog4.xes')
pm4py.write_xes(sub_log5, 'Mimic_sublogs_normalalised/sublog5.xes')
pm4py.write_xes(sub_log6, 'Mimic_sublogs_normalalised/sublog6.xes')
pm4py.write_xes(sub_log7, 'Mimic_sublogs_normalalised/sublog7.xes')
pm4py.write_xes(sub_log8, 'Mimic_sublogs_normalalised/sublog8.xes')
pm4py.write_xes(sub_log9, 'Mimic_sublogs_normalalised/sublog9.xes')
pm4py.write_xes(sub_log10, 'Mimic_sublogs_normalalised/sublog10.xes')